In [62]:
#this cell is for debug do not need to run this
# Function to convert a 32-bit binary string into 8 signed 4-bit values
def binary_to_signed_values(binary_str):
    values = []
    for i in range(0, len(binary_str), 4):
        value = int(binary_str[i:i+4], 2)
        # Convert to signed 4-bit
        if value >= 8:
            value -= 16
        values.append(value)
    return values

# Function to calculate MAC results
def compute_mac_for_all_columns(activation_line, weight_lines):
    activation_values = binary_to_signed_values(activation_line)
    results = []
    for weight_line in weight_lines:
        weight_values = binary_to_signed_values(weight_line)
        mac_result = sum(a * b for a, b in zip(activation_values, weight_values))
        # Ensure result is within 8-bit range
        
        mac_result = max(-128, min(mac_result, 127))
        # Convert MAC result to signed 8-bit binary
        binary_result = f"{(mac_result + 256) % 256:08b}"
        results.append(binary_result)
    return results

# Activation line
activation_line = "10100101100101001010010100010101"

# Weight lines
weight_lines = [
    "00000000000100011111000100000010",
    "00000000000111110001111000000000",
    "00000000000000000000000000000000",
    "00010000000100001111000011110001",
    "00001111000011110000110100000000",
    "00000000000011111111000000000000",
    "00000001000000001110001100000001",
    "00010000000111111111000000000000"
]

# Compute MAC results for the given activation and weight lines
final_results = compute_mac_for_all_columns(activation_line, weight_lines)

# Format the results as requested
output_format = "#time0col7[msb-lsb],time0col6[msb-lsb],....,time0col0[msb-lsb]#\n"
output_format += "".join(final_results[::-1])  # Reverse for col7 to col0

# Print the output
print(output_format)
print(binary_to_signed_values("01001010011110000000000010101111"))

print(binary_to_signed_values("00000000000100011111000100000010"))

#time0col7[msb-lsb],time0col6[msb-lsb],....,time0col0[msb-lsb]#
1111010100100101000000101110100011111101000000001110010100010010
[4, -6, 7, -8, 0, 0, -6, -1]
[0, 0, 1, 1, -1, 1, 0, 2]


In [45]:
#run this cell first, it generates all psum file for each kij
# Function to convert a 32-bit binary string into 8 signed 4-bit values
def binary_to_signed_values(binary_str):
    values = []
    for i in range(0, len(binary_str), 4):
        value = int(binary_str[i:i+4], 2)
        # Convert to signed 4-bit
        if value >= 8:
            value -= 16
        values.append(value)
    return values

# Function to calculate MAC results
def compute_mac_for_all_columns(activation_line, weight_lines):
    activation_values = binary_to_signed_values(activation_line)
    results = []
    for weight_line in weight_lines:
        weight_values = binary_to_signed_values(weight_line)
        mac_result = sum(a * b for a, b in zip(activation_values, weight_values))
        # Ensure result is within 8-bit range
        mac_result = max(-128, min(mac_result, 127))
        # Convert MAC result to signed 8-bit binary
        binary_result = f"{(mac_result + 256) % 256:08b}"
        results.append(binary_result)
    return results

# Read the activation file
def read_activations(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
    # Skip to the fourth line and read the activation lines
    return [line.strip() for line in lines[3:]]

# Read the weights file
def read_weights(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
    # Skip to the fourth line and read the weight lines
    return [line.strip() for line in lines[3:]]

# Main function
def main():
    # Filenames for activation file and weight files
    activation_file = 'activation.txt'
    weight_files = [f'weight_kij{i}.txt' for i in range(9)]
    output_files = [f'psum_results_kij{i}.txt' for i in range(9)]
    
    # Read activation lines
    activation_lines = read_activations(activation_file)
    
    # Header for the psum files
    header = "#time0col7[msb-lsb],time0col6[msb-lsb],....,time0col0[msb-lsb]#\n#time1col7[msb-lsb],time1col6[msb-lsb],....,time1col0[msb-lsb]#\n#................#\n"
    
    # Loop through each weight file and compute the psum results
    for weight_file, output_file in zip(weight_files, output_files):
        # Read weight lines
        weight_lines = read_weights(weight_file)
        
        # Compute MAC results for each activation line and weight matrix
        all_results = []
        for time, activation_line in enumerate(activation_lines):
            mac_results = compute_mac_for_all_columns(activation_line, weight_lines)
            # Format the results for the current time
            time_results = "".join(mac_results[::-1])  # Reverse for col7 to col0
            all_results.append(time_results)
        
        # Combine all results into the final output
        final_output = header + "\n".join(all_results)
        
        # Write the output to a file
        with open(output_file, 'w') as file:
            file.write(final_output)
        
        print(f"Results written to {output_file}")

# Run the program
if __name__ == "__main__":
    main()


Results written to psum_results_kij0.txt
Results written to psum_results_kij1.txt
Results written to psum_results_kij2.txt
Results written to psum_results_kij3.txt
Results written to psum_results_kij4.txt
Results written to psum_results_kij5.txt
Results written to psum_results_kij6.txt
Results written to psum_results_kij7.txt
Results written to psum_results_kij8.txt


In [40]:
#assume input feature map is of size a*a and kernel size is 3*3 and assume it is already zero-padded, then output should be
#(a-3)/1+1 and result map should be of b*b
#dataarray[kij][nij(refer to input featurem map)] = 
#for i from 0 to a-3: 
#    for j from 0 to a-3:            
#        for kernelsize from 0 to 8(inclusive)        
#             start_index = nij(input feature)i*a+j
#             result[i][j] += [(start_index+kernelsize/2*a+kernelsize%2)][kernelsize]

In [41]:
#run this cell next
import os
import numpy as np

# Initialize an empty list to hold the data for all files
data_array = []

# Read all psum_results_kij*.txt files
for kij in range(9):  # For kij0 to kij8
    filename = f"psum_results_kij{kij}.txt"
    with open(filename, 'r') as file:
        # Read lines and skip the first 3
        lines = file.readlines()[3:]
        # Convert the remaining lines into a list of strings
        data_array.append([line.strip() for line in lines])

# Convert data_array into a numpy array for structured indexing
data_array_np = np.array(data_array, dtype=object)

# Display the data array to the user
print(data_array_np)


[['...']]


In [42]:
#run this cell next
#for i from 0 to a-3: 
#    for j from 0 to a-3:            
#        for kernelsize from 0 to 8(inclusive)        
#             start_index = i*a+j
#             result[i][j] += data_array_np[(start_index+kernelsize/2*a+kernelsize%2)][kernelsize]
# Assuming `a` is defined and `data_array_np` has been populated from the previous step

# Initialize the result array with zeros
# import math
# print(data_array_np.shape)  # Output: (2, 3)

# a = int(math.sqrt(data_array_np.shape[1]))  # Assuming the second dimension of data_array_np gives `a`
# result = np.zeros((a - 3, a - 3))

# # Nested loops as per the given logic
# for i in range(a - 3+1):  # Outer loop
#     for j in range(a - 3+1):  # Inner loop
#         for kernelsize in range(9):  # Kernel size loop from 0 to 8
#             start_index = i * a + j
#             # Access the corresponding data from data_array_np
#             result[i][j] += data_array_np[kernelsize][(start_index + kernelsize // 3 * a + kernelsize % 3)]
import math
import numpy as np

# Placeholder for `data_array_np` since it's undefined in this context.
# Define the actual `data_array_np` based on your input for the code to execute correctly.

# Assuming data_array_np is already loaded with shape (9, N) where each entry is a 64-bit binary string.

# Define dimensions and initialize result
a = int(math.sqrt(data_array_np.shape[1]))  # Assuming the second dimension gives `a`
result = np.zeros((a - 3 + 1, a - 3 + 1), dtype=object)

# Loop through all indices
for i in range(a - 3 + 1):  # Outer loop
    for j in range(a - 3 + 1):  # Inner loop
        # Create a temporary array to hold 8 intermediate sums (one per 8-bit segment)
        intermediate_sums = np.zeros(8, dtype=int)
        for kernelsize in range(9):  # Kernel size loop from 0 to 8
            start_index = i * a + j
            # Extract the binary line corresponding to the current indices
            binary_line = data_array_np[kernelsize][(start_index + kernelsize // 3 * a + kernelsize % 3)]
            # Split the binary line into 8-bit segments and convert to signed integers
            for segment_index in range(8):
                segment_value = int(binary_line[segment_index * 8:(segment_index + 1) * 8], 2)
                # Convert to signed 8-bit value if necessary
                if segment_value >= 128:  # Check for negative numbers in 8-bit signed
                    segment_value -= 256
                intermediate_sums[segment_index] += segment_value
        # Convert the summed values back to signed 8-bit binary strings
        final_binary_result = ''.join(
            f"{(value + 256) % 256:08b}" for value in intermediate_sums
        )
        # Store the final 64-bit result in the result array
        result[i][j] = final_binary_result
print(result)

[['0001010110011101001010101100000000110001000000001011010101001110'
  '0101000101000101001101100000110001010111000000000000100100100100'
  '1111011111001001001001001001101000011110000000001000101101100011'
  '0011001101110001001100001110011001000100000000001101111100111001']
 ['1111011111001001001001001001101000011110000000001000101101100011'
  '0101000101000101001101100000110001010111000000000000100100100100'
  '0001010110011101001010101100000000110001000000001011010101001110'
  '0101000101000101001101100000110001010111000000000000100100100100']
 ['1111011111001001001001001001101000011110000000001000101101100011'
  '0011001101110001001100001110011001000100000000001101111100111001'
  '1111011111001001001001001001101000011110000000001000101101100011'
  '0101000101000101001101100000110001010111000000000000100100100100']
 ['1111011111001001001001001001101000011110000000001000101101100011'
  '0011001101110001001100001110011001000100000000001101111100111001'
  '1111011111001001001001001001

In [44]:
#note:this nij refer to output feature map nij
#this cell generate final output file
header = [
    "#nij0col7[msb-lsb],nij0col6[msb-lsb],....,nij0col0[msb-lsb]#",
    "#nij1col7[msb-lsb],nij1col6[msb-lsb],....,nij1col0[msb-lsb]#",
    "#................#"
]

# Specify the output file name
output_filename = "output.txt"

# Write the result to the output file
with open(output_filename, 'w') as file:
    # Write the header
    for line in header:
        file.write(line + "\n")
    
    # Write the result data row by row
    for row in result:
        for content in row:
            file.write(content + "\n")

print(f"Results written to {output_filename}")

Results written to output.txt
